# Describe the business problem
This notebook describes how to use CPLEX Modeling for Python to manage the production of pasta to meet demand with your resources.

The model aims at minimizing the production cost for a number of products while satisfying customer demand.

Each product can be produced either inside the company or outside, at a higher cost.
The inside production is constrained by the company's resources, while outside production is considered unlimited.
The model first declares the products and the resources. The data consists of the description of the products (the demand, the inside and outside costs, and the resource consumption) and the capacity of the various resources. The variables for this problem are the inside and outside production for each product.

# Describe the business problem - Consumer Products Distribution 
We will model distribution of consumer products. As a distributor, the profit is based on the margin and how much can be sold. There is a limited supply of products available at the supplier side and a given maximum demand from the clients.

Also, the distributor has a limited number of trucks to ship these products.The model aims at minimizing the transportation cost for a number of products while satisfying customer demand and maximizing profit margin

All this information will be supplied in the form of a set of input tables

In [39]:
%reset -f

In [40]:
import pandas as pd

The demand table provides the demand for each product and customer combination

In [ ]:
demand = inputs['Demand'].set_index(['product','customer'], verify_integrity=True).sort_index()
inventory = inputs['Inventory'].set_index('product', verify_integrity=True)
product_margin = inputs['Margin'].set_index('product', verify_integrity=True)
trucks_df = inputs['Trucks'].set_index('truck_model', verify_integrity=True)
params = inputs['Parameters'].set_index('param')

# Step 1: Pre-process the input tables and generate intermediate data
The input tables cannot directly be used to generate the optimization model. 
We will do some pre-processing to combine data to make it easier to generate the model. We typically call these tables 'intermediate' data.

Join product_margin into demand. Because of matching index names, this can be done with a simple join.
This will make it easy to express the margin objective.

*Note: we could also have merged-in the inventory-level. But as an example, we'll use a different approach to illustrate how to use the alternative.*

In [46]:
demand_margin = demand.join(product_margin)
demand_margin.head()

demand  margin  size
product            customer                      
household_supplies Store01      600    0.12   1.0
                   Store02      900    0.12   1.0
                   Store03      700    0.12   1.0
                   Store04      600    0.12   1.0
                   Store05      800    0.12   1.0

In [47]:
customers = demand.index.get_level_values('customer').unique()
trucks = trucks_df.index
idx = pd.MultiIndex.from_product([customers, trucks], names=['customer', 'truck_model'])
customer_truck = pd.DataFrame(index=idx).join(trucks_df[['truck_capacity', 'truck_cost']])
customer_truck.head()

truck_capacity  truck_cost
customer truck_model                                 
Store01  Straight_truck             10000          10
         Dryvan_trailer53           25000          17
Store02  Straight_truck             10000          10
         Dryvan_trailer53           25000          17
Store03  Straight_truck             10000          10

# Step 2: Create model

Decision variables:
* $xDeliver$ : for each customer demand, how much can I supply?
* $xNumTrucks$: number of trucks of a particular model are assigned to a client?

Objective:
* Maximize profit = revenue - cost

Constraints:
* Limited inventory
* Limited set of trucks
* Limted truck size

In [48]:
from docplex.mp.model import Model
mdl = Model(name='CPG_Distribution')

## Decision variables

Decision variables:
* $xDeliver$ : for each customer demand, how much can I supply?
* $xNumTrucks$: number of trucks of a particular model are assigned to a client?

We can use the demand as the upper-bound for the delivery dvar. This avoids creating an explicit constraint.
Note the vector representation of the upper-bound (`ub`) input variable.

Best practices:
* Use a name for the dvar: will make reading the lp file and debugging the model easy. By ensuring that the df is indexed, docplex will use the index names for the dvar name.
* Specify the lb and ub instead of using constraints.
* Assigning the dvar as a column to the dataframe. This will make it easy to create constraints by manipulating the df. Also, after the solve, we can easily extract the solution into an output df.

### Amount to deliver
$xDeliver$ : for each customer demand, how much can I supply?
$$
\begin{align}
xDeliver_{p,c} &&\qquad \forall  p \in products, c \in customers\\
\end{align}
$$

In [49]:
demand_margin['xDeliver'] = pd.Series(mdl.continuous_var_list(demand_margin.index, lb=0, ub=demand_margin.demand, name='xDeliver'), index = demand_margin.index)

In [50]:
demand_margin.head()

demand  margin  size  \
product            customer                         
household_supplies Store01      600    0.12   1.0   
                   Store02      900    0.12   1.0   
                   Store03      700    0.12   1.0   
                   Store04      600    0.12   1.0   
                   Store05      800    0.12   1.0   

                                                        xDeliver  
product            customer                                       
household_supplies Store01   xDeliver_household_supplies_Store01  
                   Store02   xDeliver_household_supplies_Store02  
                   Store03   xDeliver_household_supplies_Store03  
                   Store04   xDeliver_household_supplies_Store04  
                   Store05   xDeliver_household_supplies_Store05

### Amount of trucks assigned to client
$xNumTrucks$: number of trucks of a particular model are assigned to a client
$$
\begin{align}
xNumTrucks_{c, t} &&\qquad \forall  c \in customers, t \in trucks\\
\end{align}
$$

In [51]:
customer_truck['xNumTrucks'] = pd.Series(mdl.integer_var_list(customer_truck.index, lb=0, name='xNumTrucks'), index = customer_truck.index)
customer_truck.head()

truck_capacity  truck_cost  \
customer truck_model                                    
Store01  Straight_truck             10000          10   
         Dryvan_trailer53           25000          17   
Store02  Straight_truck             10000          10   
         Dryvan_trailer53           25000          17   
Store03  Straight_truck             10000          10   

                                                    xNumTrucks  
customer truck_model                                            
Store01  Straight_truck      xNumTrucks_Store01_Straight_truck  
         Dryvan_trailer53  xNumTrucks_Store01_Dryvan_trailer53  
Store02  Straight_truck      xNumTrucks_Store02_Straight_truck  
         Dryvan_trailer53  xNumTrucks_Store02_Dryvan_trailer53  
Store03  Straight_truck      xNumTrucks_Store03_Straight_truck

## Step 3: Create KPIs and Objective
Best practice is to define individual cost or benefit expressions as 'KPIs'. This allows the CPLEX model to report these KPIs seperately.

### Margin
$$
\begin{align}
margin &= \sum\limits_{\forall p, c} xDeliver_{p,c} * margin_p &&\qquad \text{with }  p \in product, c \in clients\\
\end{align}
$$

In [52]:
margin_kpi = mdl.sum(demand_margin.xDeliver * demand_margin.margin)
mdl.add_kpi(margin_kpi, "Margin")

DecisionKPI(name=Margin,expr=0.120xDeliver_household_supplies_Store01+0.120xDeliver_household..)

### Transportation cost
$$
\begin{align}
transportation &= \sum\limits_{\forall t} xNumTrucks_{c,t} * cost_t &&\qquad \text{with } c \in customers, t \in trucks\\
\end{align}
$$

In [53]:
transportation_kpi = mdl.sum(customer_truck.xNumTrucks * customer_truck.truck_cost)
mdl.add_kpi(transportation_kpi, "Transportation")

DecisionKPI(name=Transportation,expr=10xNumTrucks_Store01_Straight_truck+17xNumTrucks_Store01_Dryvan_..)

### Objective

In [54]:
mdl.maximize(margin_kpi - 0.1* transportation_kpi)

## Step 4: Setup Constraints

#### Inventory constraint

$$
\begin{align}
\sum\limits_{\forall c} xDeliver_{p,c}  <= inventory_p \quad \forall p \in products \text{, with } c \in customers_p\\
\end{align}
$$

In [55]:
for product, group in demand_margin.groupby('product'):
    product_inventory = inventory.at[product, 'inventory']  # Use inventory as lookup table
    mdl.add_constraint(mdl.sum(group.xDeliver) <= product_inventory, 'InventoryC_{}'.format(product))

#### Truck capacity constraint

$$
\begin{align}
\sum\limits_{\forall p} xDeliver_{p,c} * size_p<=  \sum\limits_{\forall t} xNumTrucks_{c,t} * truckCapacity_t \quad \forall c \in customers \text{, with } p \in products, t \in truckModels\\
\end{align}
$$

In [56]:
for customer, group in demand_margin.groupby('customer'):
    customer_trucks = customer_truck.query("customer == @customer")
    truck_capacity = mdl.sum(customer_trucks.xNumTrucks * customer_trucks.truck_capacity)
    mdl.add_constraint(mdl.sum(group.xDeliver * group.size) <= truck_capacity, 'TruckCapacityC_{}'.format(customer))

#### Truck availability constraint

$$
\begin{align}
\sum\limits_{\forall c} xNumTrucks_{c,t} <= truckAvailability_t \quad \forall t \in truckModels \text{, with } c \in customers\\
\end{align}
$$

In [57]:
for truck_model, group in customer_truck.groupby('truck_model'):
    truck_availability = trucks_df.at[truck_model, 'availability']
    mdl.add_constraint(mdl.sum(group.xNumTrucks) <= truck_availability, 'TruckAvailabilityC_{}'.format(customer))

## Step 5: Solve
* log_output shows cplex log
* Always add a timelimit in case of unexpected very long solves

In [58]:
msol = mdl.solve(log_output=True, cplex_parameters = {'timelimit':30})
if msol is not None:
    mdl.report()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               30
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 0 columns.
Reduced MIP has 13 rows, 60 columns, and 90 nonzeros.
Reduced MIP has 0 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 13 rows, 60 columns, and 90 nonzeros.
Reduced MIP has 0 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (0.06 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective

In [59]:
if msol is not None:
    mdl.print_solution()

objective: 3065.900
  xDeliver_household_supplies_Store01=520.000
  xDeliver_household_supplies_Store03=700.000
  xDeliver_household_supplies_Store04=600.000
  xDeliver_household_supplies_Store05=800.000
  xDeliver_household_supplies_Store06=850.000
  xDeliver_household_supplies_Store07=660.000
  xDeliver_household_supplies_Store08=500.000
  xDeliver_household_supplies_Store09=600.000
  xDeliver_household_supplies_Store10=770.000
  xDeliver_packaged_goods_Store01=900.000
  xDeliver_packaged_goods_Store02=500.000
  xDeliver_packaged_goods_Store03=700.000
  xDeliver_packaged_goods_Store04=900.000
  xDeliver_packaged_goods_Store05=600.000
  xDeliver_packaged_goods_Store06=650.000
  xDeliver_packaged_goods_Store07=770.000
  xDeliver_packaged_goods_Store08=700.000
  xDeliver_packaged_goods_Store09=900.000
  xDeliver_packaged_goods_Store10=300.000
  xDeliver_personal_care_Store01=267.500
  xDeliver_personal_care_Store02=662.500
  xDeliver_personal_care_Store03=550.000
  xDeliver_personal_car

## Step 6:  Extract solution

In [60]:
demand_margin['planned_delivery'] = [dvar.solution_value for dvar in demand_margin.xDeliver]
demand_margin_output = demand_margin.drop(columns=['xDeliver'])
demand_margin_output.head()

demand  margin  size  planned_delivery
product            customer                                        
household_supplies Store01      600    0.12   1.0             520.0
                   Store02      900    0.12   1.0               0.0
                   Store03      700    0.12   1.0             700.0
                   Store04      600    0.12   1.0             600.0
                   Store05      800    0.12   1.0             800.0

In [61]:
customer_truck['num_trucks'] = [dvar.solution_value for dvar in customer_truck.xNumTrucks]
customer_truck_output = customer_truck.drop(columns=['xNumTrucks'])
customer_truck_output.head()

truck_capacity  truck_cost  num_trucks
customer truck_model                                             
Store01  Straight_truck             10000          10         1.0
         Dryvan_trailer53           25000          17         1.0
Store02  Straight_truck             10000          10         0.0
         Dryvan_trailer53           25000          17         1.0
Store03  Straight_truck             10000          10         4.0

## Step 7: Post processing<br>
Compute backlog. Because we extracted the solution in a column, we can do vector operations on it.

In [62]:
demand_margin_output['backlog'] = (demand_margin_output.demand - demand_margin_output.planned_delivery).clip(lower=0)
demand_margin_output.head()

demand  margin  size  planned_delivery  backlog
product            customer                                                 
household_supplies Store01      600    0.12   1.0             520.0     80.0
                   Store02      900    0.12   1.0               0.0    900.0
                   Store03      700    0.12   1.0             700.0      0.0
                   Store04      600    0.12   1.0             600.0      0.0
                   Store05      800    0.12   1.0             800.0      0.0

## Step 8: Export lp file

In [63]:
mdl.export_as_lp('CPG.lp')

'CPG.lp'

In [65]:
import os
os.path
current_path = os.getcwd()
current_path

'/home/wsuser/work/project_git_repo/HandonLabWSP/assets/jupyterlab'

## Step 9: Write to csv
See next step in tutorial: reading inputs from csv

Write parameters to parameter table, so we can use it as an input

In [69]:
params = pd.DataFrame([{'param': 'truck_capacity', 'value': 1000}]).set_index('param')
params

,value
param,
truck_capacity,1000


In [70]:
demand.to_csv('/home/wsuser/work/project_data_assets/data_asset/Demand.csv')
inventory.to_csv('/home/wsuser/work/project_data_assets/data_asset/Inventory.csv')
product_margin.to_csv('/home/wsuser/work/project_data_assets/data_asset/Margin.csv')
trucks_df.to_csv('/home/wsuser/work/project_data_assets/data_asset/Trucks.csv')
params.to_csv('/home/wsuser/work/project_data_assets/data_asset/Parameters.csv')